# Création d'un modèle de prédiction de débits


In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np

df = pd.read_csv('data/donnees-hydro-meteo.csv')
df = df.dropna(thresh=len(df) - 10, axis=1) #on garde les stations avec moins de 10 jours vides
df = df.fillna(method='ffill')

In [ ]:
# Sélection des features
target_col = 'O919001001' #la Garonne à la Réole
exclure = ["O200001001",'O180405201']
df = df[df.columns.difference(exclure)]

In [ ]:
# Modèle 1, régression linéaire toute bête, sans intégrer la météo
X = df.loc[:,df.columns != target_col]
X = X.drop(columns=['Date'])
X = X[list(X.filter(regex='^[A-Za-z].*'))]
Y = df[target_col] #Tonneins

for jour in [1,2,3] : 
    model = LinearRegression().fit(X[jour:],Y[:-jour])
    pred = model.predict(X[jour:])
    plt.plot(df['Date'][:-jour],pred,'+',label='J+'+str(jour)+' , R2 = '+str(model.score(X[jour:],Y[:-jour]))[:4])

plt.plot(df['Date'],Y, label='Débit de la Garonne à la Réole')
plt.legend()
plt.xlabel('Date')
plt.xticks(np.arange(0,len(df),30),rotation=40)
plt.ylabel('Débit ')
plt.title('Prédiction de débits sur la Réole sans les données météorologiques')
plt.show()

In [ ]:
# Modèle 1, régression linéaire toute bête, en intégrant la météo

X = df.loc[:,df.columns != target_col]
X = X.drop(columns=['Date'])
Y = df[target_col] #Tonneins

for jour in [1,2,3] : 
    model = LinearRegression().fit(X[jour:],Y[:-jour])
    pred = model.predict(X[jour:])
    plt.plot(df['Date'][:-jour],pred,'+',label='J+'+str(jour)+' , R2 = '+str(model.score(X[jour:],Y[:-jour]))[:4])

plt.plot(df['Date'],Y, label='Débit de la Garonne à la Réole')
plt.legend()
plt.xlabel('Date')
plt.xticks(np.arange(0,len(df),30),rotation=40)
plt.ylabel('Débit')
plt.title('Prédiction de débits sur la Réole avec les données météorologiques')
plt.show()

In [ ]:
plt.bar(X.columns,abs(model.coef_))
plt.xticks(rotation=45)

In [ ]:
# Modèle 2, petit réseau de neurones dense avec la météo
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

X = df.loc[:,df.columns != target_col]
X = X.drop(columns=['Date'])
Y = df[target_col] #Tonneins

model = Sequential()
model.add(Dense(30))
model.add(Dense(50))
model.add(Dense(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['r2_score'])

for jour in [1,2,3] : 
    model.fit(X[jour:],Y[:-jour],epochs=200,verbose=0)
    pred = model.predict(X[jour:])
    plt.plot(df['Date'][:-jour],pred,'+',label='J+'+str(jour)+' , R2 = '+str(model.evaluate(X[jour:],Y[:-jour])[1])[:4])

plt.plot(df['Date'],Y, label='Débit de la Garonne à la Réole')
plt.legend()
plt.xlabel('Date')
plt.xticks(np.arange(0,len(df),30),rotation=40)
plt.ylabel('Débit ')
plt.title('Prédiction de débits sur la Réole avec les données météorologiques avec un petit réseau de neurones')
plt.show()

In [ ]:
# Modèle 2, petit réseau de neurones dense sans la météo
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

X = df.loc[:,df.columns != target_col]
X = X.drop(columns=['Date'])
X = X[list(X.filter(regex='^[A-Za-z].*'))]
Y = df[target_col] #Tonneins


model = Sequential()
model.add(Dense(30))
model.add(Dense(50))
model.add(Dense(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['r2_score'])

for jour in [1,2,3] : 
    model.fit(X[jour:],Y[:-jour],epochs=200,verbose=0)
    pred = model.predict(X[jour:])
    plt.plot(df['Date'][:-jour],pred,'+',label='J+'+str(jour)+' , R2 = '+str(model.evaluate(X[jour:],Y[:-jour])[1])[:4])

plt.plot(df['Date'],Y, label='Débit de la Garonne à la Réole')
plt.legend()
plt.xlabel('Date')
plt.xticks(np.arange(0,len(df),30),rotation=40)
plt.ylabel('Débit ')
plt.title('Prédiction de débits sur la Réole sans les données météorologiques avec un petit réseau de neurones')
plt.show()